In [2]:
# ----------------------------
# Check Python Environment
# ----------------------------

import sys
import importlib
import platform
from pathlib import Path

print("Checking Python environment...\n")

# ----------------------------
# Read requirements.txt
# ----------------------------
req_file = Path("requirements.txt")
if not req_file.exists():
    print(f"⚠️  requirements.txt not found at {req_file.resolve()}")
    requirements = []
else:
    with req_file.open() as f:
        requirements = [line.strip() for line in f if line.strip() and not line.startswith("#")]

# ----------------------------
# Define exceptions for import names
# ----------------------------
# By default, assume package name == import name
# Only list exceptions here
exceptions = {
    "scikit-learn": "sklearn",
    # Add more exceptions here if needed
}

# ----------------------------
# Check missing packages
# ----------------------------
missing = []
for req in requirements:
    pkg_name = req.split(">=")[0].split("<")[0].split("==")[0].strip()
    import_name = exceptions.get(pkg_name, pkg_name)
    try:
        importlib.import_module(import_name)
    except ImportError:
        missing.append(req)

if missing:
    print("⚠️  Missing required packages:\n")
    for m in missing:
        print(f"   - {m}")
else:
    print("✅ All required packages are installed!")

# ----------------------------
# Platform info
# ----------------------------
plat = platform.system()
arch = platform.machine()

print("\n🔍 Platform info:")
print(f"   OS: {plat}, Architecture: {arch}")

# ----------------------------
# Platform-aware pip install instructions
# ----------------------------
if missing:
    print("\nTo install missing packages in this notebook, run the following commands:")
    
    # Separate torch handling for Apple Silicon CPU-only
    other_packages = [m for m in missing if not m.startswith("torch")]
    torch_packages = [m for m in missing if m.startswith("torch")]

    if other_packages:
        cmd_other = "%pip install " + " ".join(f'"{m}"' for m in other_packages)
        print(cmd_other)

    if torch_packages:
        if plat == "Darwin" and arch == "arm64":
            print("\n⚠️  Apple Silicon detected. Special instructions may be needed for PyTorch:")
            print("CPU-only PyTorch install (recommended for most users):")
            cmd_torch = "%pip install " + " ".join(
                f'"{m.split()[0]} --index-url https://download.pytorch.org/whl/cpu"' for m in torch_packages
            )
            print(cmd_torch)
        else:
            cmd_torch = "%pip install " + " ".join(f'"{m}"' for m in torch_packages)
            print(cmd_torch)

# ----------------------------
# Check that repository modules are importable
# ----------------------------
print("\n🔍 Checking that repository modules are importable...\n")

repo_modules = ["src.parsing", "src.mcmc", "src.inference", "CRN_Simulation.CRN"]
errors = False
for mod in repo_modules:
    try:
        importlib.import_module(mod)
        print(f"✅ Successfully imported: {mod}")
    except ImportError as e:
        print(f"❌ Error importing {mod}: {e}")
        errors = True

if errors:
    print("\nMake sure you haven't moved folders or renamed directories.")
else:
    print("\n✅ All repository modules are importable.")


Checking Python environment...

✅ All required packages are installed!

🔍 Platform info:
   OS: Darwin, Architecture: arm64

🔍 Checking that repository modules are importable...

✅ Successfully imported: src.parsing
✅ Successfully imported: src.mcmc
✅ Successfully imported: src.inference
✅ Successfully imported: CRN_Simulation.CRN

✅ All repository modules are importable.
